# Omforma paneldata mellan long och wide med `reshape`

Paneldata kan som beskrivits [i ett annat inlägg](xtset_sv.html) struktureras på ett långt (long) eller brett (wide) sätt, beroende på syfte. Nedan visas exempel på hur de olika strukturerna ser ut. I bred data är varje variabel med flera gånger, en gång för varje tidsenhet, och en observation per panelenhet (land i det här fallet). I lång data är varje variabel bara med en gång, men varje panelenhet är med många gånger, och så anges tiden i en särskild variabel.

I den här guiden ska vi gå igenom hur man omformar datan mellan de två strukturerna, med hjälp av kommandot `reshape`.

**Bred (wide) data:**

| country | population2000  | population2001  | population2002  |
|---------|-----------------|-----------------|-----------------|
| Sweden  | 8872284         | 8888675         | 8911899         |
| Norway  | 4491572         | 4514907         | 4537240         |


**Lång (long) data:**

| country | year     | population      |
|---------|----------|-----------------|
| Sweden  | 2000     | 8872284         | 
| Sweden  | 2001     | 8888675         | 
| Sweden  | 2002     | 8911899         | 
| Norway  | 2000     | 4491572         | 
| Norway  | 2001     | 4514907         | 
| Norway  | 2002     | 4537240         |



## Omforma från lång (long) till bred (wide)
Vi laddar först in lite data att jobba med: QoG-institutets time-series cross-section dataset. Där har vi data om länder, över tid. För att det ska bli lite mer lättöverskådligt slänger vi också bort de flesta variabler. Vi behåller variablerna "cname", "year", "fh_status" (demokratistatus) och "unna_pop" (befolkningen), samt bara åren 2010 och 2011. Det gör vi med hjälp av kommandot `keep` och en [if-sats.](ifstatements_sv.html)

In [34]:
use "https://www.qogdata.pol.gu.se/dataarchive/qog_bas_ts_jan18.dta", clear
keep cname year fh_status unna_pop
keep if year==2010 | year==2011


(Quality of Government Basic dataset 2018 - Time-Series)


(14,770 observations deleted)


Om vi nu tittar på de första sex raderna i datasetet ser vi att varje land är med två gånger, en gång för 2010 och en gång för 2011. Varje variabel är bara med en gång.

In [35]:
list in 1/6


     +---------------------------------------------+
     |       cname   year     fh_status   unna_pop |
     |---------------------------------------------|
  1. | Afghanistan   2010      Not Free   27962207 |
  2. | Afghanistan   2011      Not Free   28809167 |
  3. |     Albania   2010   Partly Free    2901883 |
  4. |     Albania   2011   Partly Free    2886010 |
  5. |     Algeria   2010      Not Free   36036159 |
     |---------------------------------------------|
  6. |     Algeria   2011      Not Free   36717132 |
     +---------------------------------------------+


### Reshape-kommandot från long till wide
Nu ska vi köra själva reshape-kommandot. Där behöver vi ange några saker.
1. Om slutresultatet ska vara long eller wide.
2. Vilka variabler det är som ska omformas (fh_status och unna_pop i det här fallet)
3. Vilken variabel som anger panelenheten (cname i det här fallet)
4. Vilken variabel som anger tidsenheten (year i det här fallet)

De första två sakerna anges före kommatecknet, och de två andra i options `i()` (panelenheten) och `j()` (tidsenheten). Såhär ser det alltså ut:

In [36]:
reshape wide fh_status-unna_pop, i(cname) j(year)

(note: j = 2010 2011)

Data                               long   ->   wide
-----------------------------------------------------------------------------
Number of obs.                      422   ->     211
Number of variables                   4   ->       5
j variable (2 values)              year   ->   (dropped)
xij variables:
                              fh_status   ->   fh_status2010 fh_status2011
                               unna_pop   ->   unna_pop2010 unna_pop2011
-----------------------------------------------------------------------------


Vi får ut en liten statusrapport som visar hur det gick: Antalet observationer har minskat till hälften, vilket är som det ska - förut var ju varje land två observationer, nu är det bara en. Antalet variabler har dock blivit större. Variabeln "year" har försvunnit (den avspeglas nu i de andra variabelnamnen): fh_status har blivit till fh_status2010 och fh_status2011, medan unna_pop har blivit till unna_pop2010 och unna_pop2011. Vi kollar på de första sex raderna och ser att det fungerat.

In [37]:
list in 1/6


     +-----------------------------------------------------------------------+
     |               cname   fh_sta~2010   unn~2010   fh_sta~2011   unn~2011 |
     |-----------------------------------------------------------------------|
  1. |         Afghanistan      Not Free   27962207      Not Free   28809167 |
  2. |             Albania   Partly Free    2901883   Partly Free    2886010 |
  3. |             Algeria      Not Free   36036159      Not Free   36717132 |
  4. |             Andorra          Free      84419          Free      82326 |
  5. |              Angola      Not Free   21219954      Not Free   21942296 |
     |-----------------------------------------------------------------------|
  6. | Antigua and Barbuda          Free      87233          Free      88152 |
     +-----------------------------------------------------------------------+


### Ett möjligt problem: Dubletter (values of variable not unique)
Om det inte fungerar kan ett problem vara att det finns dubletter i panel och tidsvariablerna. Då får man felmeddelandet "values of variable year not unique within cname". Använd kommandot duplicates list för att hitta dessa dubletter: `duplicates list cname year` och se vad som blivit fel.

## Omforma från bred (wide) till lång (long)
Nu har vi alltså ett brett dataset. Om man vill omforma ett sådant till ett långt går det också bra att göra med `reshape`. Kanske är detta något vanligare om man hämtar data från källor som kanske inte är anpassade för statistisk analys.

För att göra det behöver variabelnamnen se ut på ett speciellt sätt. Alla variabler som visar samma information ska heta samma sak, fast med olika siffror på slutet. De ska alltså heta "unna_pop2010", "unna_pop2011", "unna_pop2012" osv, inte "unna2010pop", "unna2011pop". Siffrorna måste vara på slutet för att Stata ska fatta.

### Reshape-kommandot från bred (wide) till lång (long)
Kommandot är likt det vi använder när vi ska gå från lång till bred, men inte riktigt. Först skriver vi `reshape long` för att signalera att det är ett långt dataset vi vill få till. Sedan ska vi skriva själva namnen på de variabler vi vill omforma, men bara grundformen av namnet! Alltså, utan årtalet. Stata letar då efter dessa grundformer i variabellistan. Om man har variabler som heter fh_status2010 och fh_status2011 skriver man alltså bara fh_status.

Sedan ska vi i option `i()` ange panelenheten, och i option `j()` ska vi ange tidsenheten. Men observera att tidsenheten inte har en egen variabel än, vi bestämmer namnet på den här! Vi kallar variabeln som visar året för "year". Såhär blir alltså kommandot:

In [38]:
reshape long fh_status unna_pop, i(cname) j(year)

(note: j = 2010 2011)

Data                               wide   ->   long
-----------------------------------------------------------------------------
Number of obs.                      211   ->     422
Number of variables                   5   ->       4
j variable (2 values)                     ->   year
xij variables:
            fh_status2010 fh_status2011   ->   fh_status
              unna_pop2010 unna_pop2011   ->   unna_pop
-----------------------------------------------------------------------------


Här ser vi att antalet observationer har fördubblats, från 211 till 422, eftersom det var två år. Förut låg åren utspridda i kolumner, nu blir de utspridda på längden istället. Variablerna har minskat i antal från 5 till 4, och en ny variabel har skapats: year. Informationen i fh_status2010 och fh_status2011 kallas ryms nu i variabeln fh_status, och unna_pop2010 och unna_pop2011 är nu i variabeln unna_pop. Om vi tittar på de första sex raderna:

In [39]:
list in 1/6


     +---------------------------------------------+
     |       cname   year     fh_status   unna_pop |
     |---------------------------------------------|
  1. | Afghanistan   2010      Not Free   27962207 |
  2. | Afghanistan   2011      Not Free   28809167 |
  3. |     Albania   2010   Partly Free    2901883 |
  4. |     Albania   2011   Partly Free    2886010 |
  5. |     Algeria   2010      Not Free   36036159 |
     |---------------------------------------------|
  6. |     Algeria   2011      Not Free   36717132 |
     +---------------------------------------------+


## Avslutning
`reshape` kan alltså användas för att omvandla data fram och tillbaka mellan långt och brett format.

När vi går från långt till brett:<br>
`reshape long variabelnamn, i(panelvariabel) j(tidsvariabel)`

När vi går från brett till långt:<br>
`reshape wide variabelnamn, i(panelvariabel) j(ny tidsvariabel)`

För de flesta analyser är långt format att föredra, eftersom man då kan [ställa in panelstrukturen med `xtset`](xtset_sv.html) och lätt göra analyser som drar nytta av tidsformatet.